

# Drive



In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Apr  8 14:31:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    47W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import

In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 22.2 MB/s eta 0:00:00


In [ ]:
!pip install kornia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.1/628.1 KB 32.4 MB/s eta 0:00:00


In [ ]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.4/716.4 KB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 18.2 MB/s eta 0:00:00


In [ ]:
# Import libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import random
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import utils
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.optim import lr_scheduler
import cv2
import shutil
import glob
import time
import timm
import warnings
import copy
import torch.nn.init as init
import gc
import PIL
import torchvision.transforms as transforms
import kornia.augmentation as K
import pytorch_lightning as pl
import albumentations as A
import torchmetrics

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import Trainer
from timm.data import resolve_data_config
from PIL import Image
from sklearn.model_selection import StratifiedKFold
from timm.data.transforms_factory import create_transform
from sklearn.model_selection import train_test_split


warnings.filterwarnings('ignore')

# From scratch

## Library

In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# ====================================================
# Library
# ====================================================
import sys
#sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image
import albumentations as A


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

'''
from albumentations import (
    Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip,
    RandomBrightness, RandomContrast, RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout,
    IAAAdditiveGaussianNoise, Transpose
    )
'''
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import timm

from sklearn.metrics import mean_squared_error
from torch.cuda.amp import autocast, GradScaler

import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Data Loading¶


In [ ]:
# Set seed for reproducability
SEED = 42

In [ ]:
# load files
data_path = '/content/drive/MyDrive/Digital-Africa-Plantation-Counting-Challenge/'

train = pd.read_csv(data_path + 'Train.csv')
test = pd.read_csv(data_path + 'Test.csv')
sample_submission = pd.read_csv(data_path + 'SampleSubmission.csv')

# Make sure that the output directory exists and is empty
'''
output_dir = os.path.join(data_path, 'TreeImages')
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

# Unzip images
shutil.unpack_archive(data_path + 'TreeImages.zip', output_dir)
'''
# Preview train
train.head()

,ImageId,Target
0,Id_jdqw9hlv6j.png,14.0
1,Id_6xtrolmuvc.png,18.0
2,Id_2m49sj3xd9.png,0.0
3,Id_9jwg5pcnn4.png,28.0
4,Id_vnm6e8n0p3.png,21.0


## Directory settings

In [ ]:
# ====================================================
# Directory settings
# ====================================================
import os

OUTPUT_DIR = '/content/drive/MyDrive/Digital-Africa-Plantation-Counting-Challenge/Models/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

TRAIN_PATH = '/content/drive/MyDrive/Digital-Africa-Plantation-Counting-Challenge/TreeImages'

## CFG

In [ ]:
# ====================================================
# CFG
# ====================================================
class CFG:
    debug=False
    print_freq=100
    num_workers=2
    model_name= 'tf_efficientnetv2_s_in21k' #tf_efficientnetv2_xl_in21k
    size=640
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=25

    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=8 # CosineAnnealingLR
    #T_0=6 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=1e-6
    batch_size=8
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42

    target_size=1
    target_cols=['Target']
    n_fold=6
    trn_fold=[0, 1, 2, 3, 4, 5]
    train=True

if CFG.debug:
    CFG.epochs = 1
    train = train.sample(n=100, random_state=CFG.seed).reset_index(drop=True)

## Utils

In [ ]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    scores = []
    for i in range(y_true.shape[1]):
        score = mean_squared_error(y_true[:,i], y_pred[:,i], squared=False)
        scores.append(score)
    avg_score = np.mean(scores)
    return avg_score, scores


@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}] start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')


def init_logger(log_file=OUTPUT_DIR+'train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()


def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

## CV split


In [ ]:

def kfold (data):
  data['Bins'] = (data['Target']//CFG.n_fold).round()
  data['fold'] = -1
  skf = StratifiedKFold(n_splits = CFG.n_fold, shuffle=True, random_state = CFG.seed)
  for i, (_, test_index) in enumerate(skf.split(data.index, data['Bins'])):
      data.iloc[test_index, -1] = i

  data['fold'] = data['fold'].astype('int')
  data_group = data.groupby(['fold'])['Target'].agg(['mean','std','count'])

  gc.collect()
  return data,data_group
folds = train.copy()
folds, folds_group = kfold(train)

In [ ]:
folds_group

,mean,std,count
fold,,,
0,10.309880,13.627451,668
1,10.230885,13.618705,667
2,10.260870,13.614855,667


## Dataset

In [ ]:
# ====================================================
# Dataset
# ====================================================
class TrainDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['ImageId'].values
        self.labels = df[CFG.target_cols].values
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TRAIN_PATH}/{file_name}'
        #print(file_path)
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']


        #image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        #label = torch.tensor(self.labels[idx]).float(32)

        #features = self.dense_features[item, :]
        label = self.labels[idx]
        image = torch.tensor(image, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.float32)



        return image, label

## Transforms

In [ ]:
# ====================================================
# Transforms
# ====================================================


def get_transforms(*, data):

    if data == 'train':
        return A.Compose([
                #A.Blur(p=0.5),
                A.CLAHE(p=0.8),
                #A.HueSaturationValue(p=0.5),
                #A.RandomBrightnessContrast(p=0.5),
                A.Sharpen(p=0.8),
                A.GaussianBlur(p=0.2),
                A.MedianBlur(p=0.2),
                A.Resize(CFG.size, CFG.size),
                #A.RandomResizedCrop(CFG.size, CFG.size, scale=(0.9, 1.0)),
                A.HorizontalFlip(p=0.5),
                A.Normalize(
                        mean=[0.03993845, 0.03993845, 0.03993845],
                        std=[0.17816752, 0.20023753, 0.14269611],
                ),
                ToTensorV2(),
            ])

    elif data == 'valid':
        return A.Compose([
            A.Resize(CFG.size, CFG.size),
            A.Normalize(
                    mean=[0.03993845, 0.03993845, 0.03993845],
                    std=[0.17816752, 0.20023753, 0.14269611],
            ),
            ToTensorV2(),
        ])




In [ ]:
train_dataset = TrainDataset(train, transform=get_transforms(data='train'))

for i in range(7):
    image, label = train_dataset[i]
    plt.imshow(image[0])
    plt.title(f'label: {label}')
    plt.show()

## MODEL

In [ ]:

class CustomModel(nn.Module):
    def __init__(self, model_name='tf_efficientnetv2_xl_in21k', pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, CFG.target_size)

    def forward(self, x):
        x = self.model(x)
        return x

## Helper functions

In [ ]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device):
    scaler = GradScaler()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to train mode
    model.train()
    start = end = time.time()
    global_step = 0
    for step, (images, labels) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)

        y_preds = model(images)
        loss = criterion(y_preds, labels)
        # record loss
        losses.update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        scaler.scale(loss).backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  #'LR: {lr:.6f}  '
                  .format(
                   epoch+1, step, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(train_loader)),
                   grad_norm=grad_norm,
                   #lr=scheduler.get_lr()[0],
                   ))
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to evaluation mode
    model.eval()
    preds = []
    start = end = time.time()
    for step, (images, labels) in enumerate(valid_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        # compute loss

        y_preds = model(images)
        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)
        # record accuracy
        #preds.append(y_preds.sigmoid().to('cpu').numpy())
        preds.append(y_preds.to('cpu').detach().numpy())
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(
                   step, len(valid_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(valid_loader)),
                   ))
    predictions = np.concatenate(preds)
    return losses.avg, predictions

## Train loop


In [ ]:
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()

    def forward(self, yhat, y):
        return torch.sqrt(self.mse(yhat, y))


In [ ]:
# ====================================================
# Train loop
# ====================================================
def train_loop(folds, fold):

    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index

    train_folds = folds.loc[trn_idx].reset_index(drop=True)
    valid_folds = folds.loc[val_idx].reset_index(drop=True)
    valid_labels = valid_folds[CFG.target_cols].values

    train_dataset = TrainDataset(train_folds,
                                 transform=get_transforms(data='train'))
    valid_dataset = TrainDataset(valid_folds,
                                 transform=get_transforms(data='valid'))

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(optimizer):
        if CFG.scheduler=='ReduceLROnPlateau':
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=CFG.factor, patience=CFG.patience, verbose=True, eps=CFG.eps)
        elif CFG.scheduler=='CosineAnnealingLR':
            scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)
        elif CFG.scheduler=='CosineAnnealingWarmRestarts':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=1, eta_min=CFG.min_lr, last_epoch=-1)
        return scheduler

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG.model_name, pretrained=True)
    model.to(device)

    ## replace AdamW with Adafactor




    #optimizer = torch.optim.NAdam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
    #optimizer = Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, amsgrad=False)
    #optimizer = torch.optim.RAdam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)

    optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
    #optimizer = torch.optim.SGD(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, momentum=0.9)

    scheduler = get_scheduler(optimizer)

    # ====================================================
    # loop
    # ====================================================
    criterion = RMSELoss()

    best_score = 0.
    best_loss = np.inf

    for epoch in range(CFG.epochs):

        start_time = time.time()

        # train
        avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)

        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(avg_val_loss)
        elif isinstance(scheduler, CosineAnnealingLR):
            scheduler.step()
        elif isinstance(scheduler, CosineAnnealingWarmRestarts):
            scheduler.step()

        # scoring
        score, scores = get_score(valid_labels, preds)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}  Scores: {np.round(scores, decimals=4)}')

        """
        if score > best_score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'preds': preds},
                        OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best.pth')
        """

        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            LOGGER.info(f'Epoch {epoch+1} - Save Best Loss: {best_loss:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'preds': preds},
                        OUTPUT_DIR+f'{CFG.model_name}_fold_v1_{fold}_best.pth')

    check_point = torch.load(OUTPUT_DIR+f'{CFG.model_name}_fold_v1_{fold}_best.pth')
    for c in [f'pred_{c}' for c in CFG.target_cols]:
        valid_folds[c] = np.nan
    valid_folds[[f'pred_{c}' for c in CFG.target_cols]] = check_point['preds']

    return valid_folds

In [ ]:
# ====================================================
# main
# ====================================================
def main():

    """
    Prepare: 1.train  2.folds
    """

    def get_result(result_df):
        preds = result_df[[f'pred_{c}' for c in CFG.target_cols]].values
        labels = result_df[CFG.target_cols].values
        score, scores = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.4f}  Scores: {np.round(scores, decimals=4)}')

    if CFG.train:
        # train
        oof_df = pd.DataFrame()
        i = 0
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(folds, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
            i = i + 1
            oof_df.to_csv(OUTPUT_DIR+'oof_df'+str(i)+'.csv', index=False)

        # CV result
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        # save result
        oof_df.to_csv(OUTPUT_DIR+'v1_oof_df.csv', index=False)

In [ ]:
if __name__ == '__main__':
    main()

========== fold: 0 training ==========
INFO:__main__:========== fold: 0 training ==========
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-effv2-weights/tf_efficientnetv2_xl_in21k-fd7e8abf.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnetv2_xl_in21k-fd7e8abf.pth


Epoch: [1][0/166] Data 20.038 (20.038) Elapsed 0m 22s (remain 61m 43s) Loss: 5.3005(5.3005) Grad: 2349952.7500  
Epoch: [1][100/166] Data 0.919 (1.130) Elapsed 2m 24s (remain 1m 32s) Loss: 5.6868(7.8634) Grad: 2041232.1250  
Epoch: [1][165/166] Data 0.000 (1.045) Elapsed 3m 41s (remain 0m 0s) Loss: 5.4720(6.6217) Grad: 1452063.7500  
EVAL: [0/84] Data 1.441 (1.441) Elapsed 0m 1s (remain 2m 8s) Loss: 6524.6104(6524.6104) 


Epoch 1 - avg_train_loss: 6.6217  avg_val_loss: 88.1294  time: 319s
INFO:__main__:Epoch 1 - avg_train_loss: 6.6217  avg_val_loss: 88.1294  time: 319s
Epoch 1 - Score: 715.7103  Scores: [715.7103]
INFO:__main__:Epoch 1 - Score: 715.7103  Scores: [715.7103]
Epoch 1 - Save Best Loss: 88.1294 Model
INFO:__main__:Epoch 1 - Save Best Loss: 88.1294 Model


EVAL: [83/84] Data 2.057 (1.077) Elapsed 1m 37s (remain 0m 0s) Loss: 2.8432(88.1294) 
Epoch: [2][0/166] Data 0.652 (0.652) Elapsed 0m 1s (remain 2m 46s) Loss: 4.2089(4.2089) Grad: 1457655.0000  
Epoch: [2][100/166] Data 0.000 (0.007) Elapsed 0m 29s (remain 0m 18s) Loss: 9.5093(3.9282) Grad: 2151872.7500  
Epoch: [2][165/166] Data 0.000 (0.004) Elapsed 0m 47s (remain 0m 0s) Loss: 3.9112(3.7484) Grad: 1980993.7500  
EVAL: [0/84] Data 0.304 (0.304) Elapsed 0m 0s (remain 0m 34s) Loss: 1218.4224(1218.4224) 


Epoch 2 - avg_train_loss: 3.7484  avg_val_loss: 23.8599  time: 58s
INFO:__main__:Epoch 2 - avg_train_loss: 3.7484  avg_val_loss: 23.8599  time: 58s
Epoch 2 - Score: 147.0390  Scores: [147.039]
INFO:__main__:Epoch 2 - Score: 147.0390  Scores: [147.039]
Epoch 2 - Save Best Loss: 23.8599 Model
INFO:__main__:Epoch 2 - Save Best Loss: 23.8599 Model


...

Epoch 24 - avg_train_loss: 1.0186  avg_val_loss: 2.2835  time: 60s
INFO:__main__:Epoch 24 - avg_train_loss: 1.0186  avg_val_loss: 2.2835  time: 60s
Epoch 24 - Score: 4.4456  Scores: [4.4456]
INFO:__main__:Epoch 24 - Score: 4.4456  Scores: [4.4456]
Epoch 24 - Save Best Loss: 2.2835 Model
INFO:__main__:Epoch 24 - Save Best Loss: 2.2835 Model


EVAL: [83/84] Data 0.000 (0.039) Elapsed 0m 10s (remain 0m 0s) Loss: 3.8690(2.2835) 
Epoch: [25][0/166] Data 1.250 (1.250) Elapsed 0m 1s (remain 4m 27s) Loss: 1.0343(1.0343) Grad: 2076966.7500  
Epoch: [25][100/166] Data 0.000 (0.013) Elapsed 0m 30s (remain 0m 19s) Loss: 1.1340(0.8143) Grad: 1540554.2500  
Epoch: [25][165/166] Data 0.000 (0.008) Elapsed 0m 49s (remain 0m 0s) Loss: 0.4426(0.9760) Grad: 1080068.5000  
EVAL: [0/84] Data 0.349 (0.349) Elapsed 0m 0s (remain 0m 39s) Loss: 1.2169(1.2169) 


Epoch 25 - avg_train_loss: 0.9760  avg_val_loss: 2.6142  time: 60s
INFO:__main__:Epoch 25 - avg_train_loss: 0.9760  avg_val_loss: 2.6142  time: 60s
Epoch 25 - Score: 5.5972  Scores: [5.5972]
INFO:__main__:Epoch 25 - Score: 5.5972  Scores: [5.5972]


EVAL: [83/84] Data 0.000 (0.041) Elapsed 0m 10s (remain 0m 0s) Loss: 3.9942(2.6142) 


========== fold: 2 result ==========
INFO:__main__:========== fold: 2 result ==========
Score: 4.4456  Scores: [4.4456]
INFO:__main__:Score: 4.4456  Scores: [4.4456]
========== CV ==========
INFO:__main__:========== CV ==========
Score: 3.2331  Scores: [3.2331]
INFO:__main__:Score: 3.2331  Scores: [3.2331]


# Inference

In [ ]:
# ====================================================
# Dataset
# ====================================================
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['ImageId'].values
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TRAIN_PATH}/{file_name}'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        image = torch.tensor(image, dtype=torch.float32)

        return image

In [ ]:
test_dataset = TestDataset(test, transform=get_transforms(data='valid'))

for i in range(5):
    image = test_dataset[i]
    plt.imshow(image[0])
    plt.show()
    plt.imshow(image[0].flip(-1))
    plt.show()

In [ ]:
def inference(models, test_loader, device):
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    preds = []
    for i, (images) in tk0:
        images = images.to(device)
        avg_preds = []
        for model in models:
            with torch.no_grad():
                y_preds1 = model(images)
                #y_preds2 = model(images.flip(-1))
            y_preds = (y_preds1.to('cpu').numpy())
            #y_preds = (y_preds1.to('cpu').numpy() + y_preds2.to('cpu').numpy()) /2

            avg_preds.append(y_preds)
        avg_preds = np.mean(avg_preds, axis=0)
        preds.append(avg_preds)
    preds = np.concatenate(preds)
    return preds

In [ ]:
model_path = ['/content/drive/MyDrive/Digital-Africa-Plantation-Counting-Challenge/Models/tf_efficientnetv2_xl_in21k_fold_v1_0_best.pth',
              '/content/drive/MyDrive/Digital-Africa-Plantation-Counting-Challenge/Models/tf_efficientnetv2_xl_in21k_fold_v1_1_best.pth']
test_dataset = TestDataset(test, transform=get_transforms(data='valid'))
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True)
e = 0
model = CustomModel( model_name='tf_efficientnetv2_xl_in21k', pretrained=False)
for i in model_path:
  e = e + 1
  model.load_state_dict(torch.load(i)['model'])
  model.eval()
  models = [model.to(device)]
  predictions = inference(models, test_loader, device)
  test[CFG.target_cols] = predictions
  test[['ImageId'] + CFG.target_cols].to_csv(OUTPUT_DIR+'TFV3'+str(e)+'Submission.csv', index=False)


  0%|          | 0/108 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

In [ ]:
ypredtest1 = pd.read_csv('/content/drive/MyDrive/Digital-Africa-Plantation-Counting-Challenge/Combine/SSSubmission (1).csv')
ypredtest2 = pd.read_csv('/content/drive/MyDrive/Digital-Africa-Plantation-Counting-Challenge/Combine/SubmissionFold5 (1).csv')
#ypredtest3 = pd.read_csv('/content/drive/MyDrive/Digital-Africa-Plantation-Counting-Challenge/Combine/tf_1Submission-2 (3).csv')
#ypredtest4 = pd.read_csv('/content/drive/MyDrive/Digital-Africa-Plantation-Counting-Challenge/Combine/submissionfoldcombine-4 (1).csv')


In [ ]:
test['Target'] =  0.5*ypredtest1.Target + 0.5*ypredtest1.Target
#test["Target"] = test["Target"].apply(lambda x: round(x, 0))
test.to_csv(OUTPUT_DIR+'FinalSubmit.csv', index=False)
test

In [ ]:
ypredtest1 = pd.read_csv('/content/drive/MyDrive/Digital-Africa-Plantation-Counting-Challenge/Combine/FinalSubmit-9 (1).csv')
ypredtest1['Target'] = ypredtest1['Target'].astype(int)

In [ ]:
ypredtest1.to_csv('FinalSubmit.csv', index=False)
